This function performs one integration step, when calculating the tensorlines (stretchlines/shrinklines) of an eigenvector field. The integration scheme is RK4.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid |
| Y | array (Ny, Nx) | Y-meshgrid |
| defined domain | boolean array (Ny, Nx) | meshgrid which denotes the domain where the velocity field is defined.|
| x | array (2,) | tensorline position |
| x_prime | array (2,) | eigenvector at point 'x' (=tangent to tensorline)|
| ds | float | fixed step-size for integration of tensorlines|
| vector_field | array(Ny, Nx) | eigenvector field (not oriented)|
| Interp_lambda | object | Interpolant of lambda|
| x_update | array (2,) | updated tensorline posiion |
| x_prime_update | array (2,) | updated eigenvector at point 'x_update' with the same orientation as 'x_prime'|

In [3]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add 'utils' folder to current working path
sys.path.append(parent_directory+"/Src/utils")

In [ ]:
# Import numpy
import numpy as np

# Import function which checks if particle is still in defined domain
from ipynb.fs.defs.check_location import check_location

# Import function which computes the scaled (and oriented) eigenvector
from ipynb.fs.defs.scaling_vectorfield import _scaling_vectorfield

In [2]:
def _RK4_tensorlines(X, Y, defined_domain, x, x_prime, ds, vector_field, Interp_eig):
        
    # Define starting point.
    x1 = x
        
    # check if point is in defined domain (given by initial meshgrid)
    loc = check_location(X, Y, defined_domain, x1)[0]
    
    # if point is not "IN" defined domain --> stop integration
    if loc != "IN":
        return None, None
        
    # Compute x_prime at the beginning of the time-step
    x_prime = _scaling_vectorfield(X, Y, x1, x_prime, vector_field, Interp_eig)
    if x_prime is None:
        return None, None
    k1 = ds * x_prime

    # Update position at the first point.
    x2 = x1 + .5 * k1
    
    # check if point is in defined domain (given by initial meshgrid)
    loc = check_location(X, Y, defined_domain, x2)[0]
    
    # if point is not "IN" defined domain --> stop integration
    if loc != "IN":
        return None, None
        
    # Compute x_prime at the first midpoint.
    x_prime = _scaling_vectorfield(X, Y, x2, x_prime, vector_field, Interp_eig)
    if x_prime is None:
        return None, None
    k2 = ds * x_prime

    # Update position at the first midpoint midpoint.
    x3 = x1 + .5 * k2
    
    # check if point is in defined domain (given by initial meshgrid)
    loc = check_location(X, Y, defined_domain, x3)[0]
    
    # if point is not "IN" defined domain --> stop integration
    if loc != "IN": 
        return None, None
    
    # Compute velocity at the second midpoint.
    x_prime = _scaling_vectorfield(X, Y, x3, x_prime, vector_field, Interp_eig)
    if x_prime is None:
        return None, None
    k3 = ds * x_prime
    
    # Update position at the second midpoint.
    x4 = x1 + k3
    
    # check if point is in defined domain (given by initial meshgrid)
    loc = check_location(X, Y, defined_domain, x4)[0]
    
    # if point is not "IN" defined domain --> stop integration
    if loc != "IN":
        return None, None
    
    # Compute velocity at the end of the time-step.
    x_prime = _scaling_vectorfield(X, Y, x4, x_prime, vector_field, Interp_eig) 
    if x_prime is None:
        return None, None
    k4 = ds * x_prime
    
    # define list for velocity and positions of particle
    x_prime_update = []
    x_update = []
        
    # Compute velocity
    for j in range(2):
        # Update velocity of particles
        x_prime_update.append(1.0 / 6.0*(k1[j] + 2 * k2[j] + 2 * k3[j] + k4[j])/ds)
    
    # Integration x <-- x + x_prime*ds
    for j in range(2):
        # Update position of particles
        x_update.append(x[j] + x_prime_update[j]*ds)

    x_update = np.array(x_update)
    x_prime_update = np.array(x_prime_update)
    
    # Update position
    if check_location(X, Y, defined_domain, x_update)[0] != "IN": 
        return None, None
    
    return x_update, x_prime_update